In [1]:
#In this script we will create star schema tables (fact table and dimensional tables) of the dataset
#we will save the tables data to csv files

In [1]:
#We will concatenate the 3 slices of the cleaned data
import pandas as pd
data1=pd.read_csv('Amazon-Sale-Report-0.csv')
data2=pd.read_csv('Amazon-Sale-Report-1.csv')
data3=pd.read_csv('Amazon-Sale-Report-2.csv')
cleaned_data=pd.concat([data1,data2,data3])
cleaned_data.shape

(122073, 22)

In [3]:
#Creating an ID column for indices (indexes)
ID=cleaned_data.index+1
ID

Int64Index([    1,     2,     3,     4,     5,     6,     7,     8,     9,
               10,
            ...
            38382, 38383, 38384, 38385, 38386, 38387, 38388, 38389, 38390,
            38391],
           dtype='int64', length=122073)

In [243]:
#Creating the dim_fulfilment table
dim_fulfilment=pd.DataFrame({'fulfilment_id':cleaned_data.index+1,
                             'fulfilment':cleaned_data['Fulfilment'],
                             'sales_channel':cleaned_data['Sales_Channel'],
                             'ship-service-level':cleaned_data['ship-service-level'],
                             'courier-status':cleaned_data['Courier_Status'],
                             'B2B':cleaned_data['B2B']}
                           )
dim_fulfilment.head()

,fulfilment_id,fulfilment,sales_channel,ship-service-level,courier-status,B2B
0,1,Merchant,Amazon.in,Standard,Shipped,False
1,2,Amazon,Amazon.in,Expedited,Shipped,True
2,3,Amazon,Amazon.in,Expedited,Shipped,False
3,4,Amazon,Amazon.in,Expedited,Shipped,False
4,5,Amazon,Amazon.in,Expedited,Shipped,False


In [231]:
#Creating dim_product table
dim_product=pd.DataFrame({'product_id':ID,
                           'style':cleaned_data['Style'],
                           'sku':cleaned_data['SKU'],
                           'category':cleaned_data['Category'],
                           'size':cleaned_data['Size'],
                           'asin':cleaned_data['ASIN']})
dim_location.head()

,location_id,ship_city,ship_state,ship_postal_code
0,1,BENGALURU,KARNATAKA,560085
1,2,NAVI MUMBAI,MAHARASHTRA,410210
2,3,CHENNAI,TAMIL NADU,600073
3,4,GHAZIABAD,UTTAR PRADESH,201102
4,5,CHANDIGARH,CHANDIGARH,160036


In [17]:
cleaned_data.columns

Index(['Order_ID', 'Date', 'Status', 'Fulfilment', 'Sales_Channel',
       'ship-service-level', 'Style', 'SKU', 'Category', 'Size', 'ASIN',
       'Courier_Status', 'Qty', 'currency', 'Amount', 'ship-city',
       'ship-state', 'ship-postal-code', 'ship-country', 'promotion-ids',
       'B2B', 'fulfilled-by'],
      dtype='object')

In [27]:
#Creating dim_date table
print(cleaned_data['Date'].unique())
print(len(cleaned_data['Date'].unique()))

['2022-04-30' '2022-04-29' '2022-04-28' '2022-04-27' '2022-04-26'
 '2022-04-25' '2022-04-24' '2022-04-23' '2022-04-22' '2022-04-21'
 '2022-04-20' '2022-04-19' '2022-04-18' '2022-04-17' '2022-04-16'
 '2022-04-15' '2022-04-14' '2022-04-13' '2022-04-12' '2022-04-11'
 '2022-04-10' '2022-04-09' '2022-04-08' '2022-04-07' '2022-04-06'
 '2022-04-05' '2022-04-04' '2022-04-03' '2022-04-02' '2022-04-01'
 '2022-03-31' '2022-05-31' '2022-05-30' '2022-05-29' '2022-05-28'
 '2022-05-27' '2022-05-26' '2022-05-25' '2022-05-24' '2022-05-23'
 '2022-05-22' '2022-05-21' '2022-05-20' '2022-05-19' '2022-05-18'
 '2022-05-17' '2022-05-16' '2022-05-15' '2022-05-14' '2022-05-13'
 '2022-05-12' '2022-05-11' '2022-05-10' '2022-05-09' '2022-05-08'
 '2022-05-07' '2022-05-06' '2022-05-05' '2022-05-04' '2022-05-03'
 '2022-05-02' '2022-05-01' '2022-06-29' '2022-06-28' '2022-06-27'
 '2022-06-26' '2022-06-25' '2022-06-24' '2022-06-23' '2022-06-22'
 '2022-06-21' '2022-06-20' '2022-06-19' '2022-06-18' '2022-06-17'
 '2022-06-

In [165]:
import numpy as np
date_id=np.array(range(1,92))
date=cleaned_data['Date'].unique()
import numpy as np
date.dtype

dtype('O')

In [167]:
dates=np.char.split(date.astype(str),sep="-")

In [171]:
dates=np.array(dates.tolist())
dates

array([['2022', '04', '30'],
       ['2022', '04', '29'],
       ['2022', '04', '28'],
       ['2022', '04', '27'],
       ['2022', '04', '26'],
       ['2022', '04', '25'],
       ['2022', '04', '24'],
       ['2022', '04', '23'],
       ['2022', '04', '22'],
       ['2022', '04', '21'],
       ['2022', '04', '20'],
       ['2022', '04', '19'],
       ['2022', '04', '18'],
       ['2022', '04', '17'],
       ['2022', '04', '16'],
       ['2022', '04', '15'],
       ['2022', '04', '14'],
       ['2022', '04', '13'],
       ['2022', '04', '12'],
       ['2022', '04', '11'],
       ['2022', '04', '10'],
       ['2022', '04', '09'],
       ['2022', '04', '08'],
       ['2022', '04', '07'],
       ['2022', '04', '06'],
       ['2022', '04', '05'],
       ['2022', '04', '04'],
       ['2022', '04', '03'],
       ['2022', '04', '02'],
       ['2022', '04', '01'],
       ['2022', '03', '31'],
       ['2022', '05', '31'],
       ['2022', '05', '30'],
       ['2022', '05', '29'],
       ['2022'

In [173]:
year=dates[:,0]
month=dates[:,1]
day=dates[:,2]

In [175]:
print(f"Year array: {year}")
print(f"Month array: {month}")
print(f"day array: {day}")

Year array: ['2022' '2022' '2022' '2022' '2022' '2022' '2022' '2022' '2022' '2022'
 '2022' '2022' '2022' '2022' '2022' '2022' '2022' '2022' '2022' '2022'
 '2022' '2022' '2022' '2022' '2022' '2022' '2022' '2022' '2022' '2022'
 '2022' '2022' '2022' '2022' '2022' '2022' '2022' '2022' '2022' '2022'
 '2022' '2022' '2022' '2022' '2022' '2022' '2022' '2022' '2022' '2022'
 '2022' '2022' '2022' '2022' '2022' '2022' '2022' '2022' '2022' '2022'
 '2022' '2022' '2022' '2022' '2022' '2022' '2022' '2022' '2022' '2022'
 '2022' '2022' '2022' '2022' '2022' '2022' '2022' '2022' '2022' '2022'
 '2022' '2022' '2022' '2022' '2022' '2022' '2022' '2022' '2022' '2022'
 '2022']
Month array: ['04' '04' '04' '04' '04' '04' '04' '04' '04' '04' '04' '04' '04' '04'
 '04' '04' '04' '04' '04' '04' '04' '04' '04' '04' '04' '04' '04' '04'
 '04' '04' '03' '05' '05' '05' '05' '05' '05' '05' '05' '05' '05' '05'
 '05' '05' '05' '05' '05' '05' '05' '05' '05' '05' '05' '05' '05' '05'
 '05' '05' '05' '05' '05' '05' '06' '06' '0

In [179]:
#Now we have evrything to build our dim_date table
dim_date=pd.DataFrame({'date_id':date_id,
                       'date':date,
                       'day':day,
                       'month':month,
                       'year':year})
dim_date

,date_id,date,day,month,year
0,1,2022-04-30,30,04,2022
1,2,2022-04-29,29,04,2022
2,3,2022-04-28,28,04,2022
3,4,2022-04-27,27,04,2022
4,5,2022-04-26,26,04,2022
...,...,...,...,...,...
86,87,2022-06-05,05,06,2022
87,88,2022-06-04,04,06,2022
88,89,2022-06-03,03,06,2022
89,90,2022-06-02,02,06,2022


In [235]:
#Creating dim_location table
dim_location=pd.DataFrame({'location_id':ID,
                           'ship_city':cleaned_data['ship-city'],
                           'ship_state':cleaned_data['ship-state'],
                           'ship_postal_code':cleaned_data['ship-postal-code']})

In [247]:
#Creating fact_orders table
fact_orders=pd.DataFrame({'order_id':cleaned_data['Order_ID'],
                          'product_id':dim_product['product_id'],
                          'location_id':dim_location['location_id'],
                          'fulfilment_id':dim_fulfilment['fulfilment_id'],
                          'qyt':cleaned_data['Qty'],
                          'amount':cleaned_data['Amount']})

In [ ]:
#Star Schema still in progress